# Imports

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from datetime import date

# Load Precipitation Data

In [2]:
filepath = 'data/ky_prec_with_gps.csv'

cols = ['STATION', 'DATE', 'PRCP', 'LATITUDE', 'LONGITUDE']

dtypes = {
    'STATION': 'category',
    'DATE': 'string',
    'PRCP': 'float32',
    'LATITUDE': 'float32',
    'LONGITUDE': 'float32',
}

weather_df = pd.read_csv(filepath, dtype=dtypes, usecols=cols)

weather_df['DATE'] = pd.to_datetime(pd.Series(weather_df['DATE']), format='%Y-%m-%d', errors='coerce').dt.date
weather_df['PRCP'] = weather_df['PRCP'].fillna(0)
weather_df['PRCP'] = weather_df.PRCP.apply(lambda x: x * 100).apply(np.int32)

display(weather_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1955383 entries, 0 to 1955382
Data columns (total 5 columns):
 #   Column     Dtype   
---  ------     -----   
 0   STATION    category
 1   LATITUDE   float32 
 2   LONGITUDE  float32 
 3   DATE       object  
 4   PRCP       int32   
dtypes: category(1), float32(2), int32(1), object(1)
memory usage: 41.1+ MB


None

In [3]:
display(weather_df.head(1))

,STATION,LATITUDE,LONGITUDE,DATE,PRCP
0,US1KYMS0002,38.532425,-83.754707,2009-04-17,0


# Load FEMA Project Data

In [4]:
filepath2 = 'data/PublicAssistanceFundedProjectsDetails.csv'

cols=[
    'disasterNumber',
    'stateCode',
    'county',
    'projectSize',
    'dcc',             # Determines if damage is Roads & Bridges
    'projectAmount'
]


dtypes={
    'disasterNumber': 'category',
    'county': 'category',
    'projectSize': 'category',
    'dcc': 'category',
    'stateCode': 'category',
    'projectAmount': 'float32'
}

df2 = pd.read_csv(filepath2, usecols=cols, dtype=dtypes)

filtered_df = df2.loc[(df2.stateCode == 'KY') & (df2.dcc == 'C')]

final_data = filtered_df.drop(labels=['stateCode', 'dcc',], axis=1)

display(final_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 15139 entries, 59 to 807119
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   disasterNumber  15139 non-null  category
 1   projectSize     15139 non-null  category
 2   county          15137 non-null  category
 3   projectAmount   15139 non-null  float32 
dtypes: category(3), float32(1)
memory usage: 416.1 KB


None

# Geopandas

In [5]:
county_shp_filename = 'zip:data/cb_2018_us_county_500k.zip'
gdf_counties = gpd.read_file(county_shp_filename)

In [6]:
stations_list = 'data/ghcnd-stations.csv'

cols=[
    'STATION',
    'LATITUDE',
    'LONGITUDE',
]

dtypes = {
    'STATION': 'category',
    'LATITUDE': 'float32',
    'LONGITUDE': 'float32',
}

stations_df = pd.read_csv(stations_list, dtype=dtypes, usecols=cols)
# display(stations_df.head(5))

In [7]:
gps = (38.532426, -83.754705)
point = Point(gps[1], gps[0])

county = gdf_counties[gdf_counties.geometry.contains(point)]
display(county)

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
3125,21,161,00516924,0500000US21161,21161,Mason,06,621924068,16281864,"POLYGON ((-83.9916 38.5937, -83.96257 38.65396..."


In [8]:
# def get_gps_from_station(name):
#     station = stations_df[stations_df['STATION'] == name]
#     return (station['LATITUDE'], station['LONGITUDE'])

def get_county_from_gps(lat, long):
    point = Point(long, lat)
    county = gdf_counties[gdf_counties.geometry.contains(point)]
    return str(county['NAME'])

# Shorthand for the 2 above functions
def get_county_from_station(name):
    gps = get_gps_from_station(name)
    county = get_county_from_gps(gps[0], gps[1])
    return county

In [9]:
# Parital Test of grabbing county for stations
# part_weather_df = weather_df[weather_df['DATE'] >= date(2025,3,1)]
# counties = part_weather_df.apply(lambda x: get_county_from_gps(x['LATITUDE'], x['LONGITUDE']), axis=1)
# display(counties)

# Full
# weather_df['COUNTY'] = weather_df.apply(lambda x: get_county_from_gps(x['LATITUDE'], x['LONGITUDE']), axis=1)
# display(weather_df.info())

# Can probably be improved by doing only getting counties for unique stations then

df_unique_stations = weather_df[['STATION', 'LATITUDE', 'LONGITUDE']].drop_duplicates()

df_unique_stations['COUNTY'] = df_unique_stations.apply(lambda x: get_county_from_gps(x['LATITUDE'], x['LONGITUDE']), axis=1).astype('category')

weather_df_with_county = weather_df.merge(
    df_unique_stations[['STATION', 'COUNTY']],
    on='STATION',
    how='left'
)

display(weather_df_with_county.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1955383 entries, 0 to 1955382
Data columns (total 6 columns):
 #   Column     Dtype   
---  ------     -----   
 0   STATION    category
 1   LATITUDE   float32 
 2   LONGITUDE  float32 
 3   DATE       object  
 4   PRCP       int32   
 5   COUNTY     category
dtypes: category(2), float32(2), int32(1), object(1)
memory usage: 44.8+ MB


None